In [6]:
import os
from typing import List,Dict,Any
import pandas as pd


In [8]:
from langchain_core.documents import Document
from langchain.text_splitter import(
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)

print("set up completed")

set up completed


In [13]:
# create simple document

doc = Document(
    page_content="You have the right to perform your duty, but never to the fruits of your actions.",
    metadata={
        "source":"mahabharat.txt",
        "pages":256,
        "author":"valmiki",
        "date_created":"1002-1-1",


    }
)

print(f"content : {doc.page_content}")
print(f"content : {doc.metadata}")


content : You have the right to perform your duty, but never to the fruits of your actions.
content : {'source': 'mahabharat.txt', 'pages': 256, 'author': 'valmiki', 'date_created': '1002-1-1'}


In [ ]:
import os
# making folder to store text files
os.makedirs("data/text_files",exist_ok=True)

insert data into text file

In [5]:
sample_data = {
    "data/text_files/python_intro":"""Python is a high-level, interpreted, object-oriented programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

⚡ Key Features of Python

Simple & Readable – Easy syntax, close to English.

Interpreted – No need to compile; runs line by line.

Cross-Platform – Works on Windows, macOS, Linux, etc.

Object-Oriented – Supports classes, objects, inheritance, etc.

Dynamic Typing – No need to declare variable types.

Large Standard Library – Comes with modules for math, OS, web, AI, etc.

Community Support – Huge developer community and resources.

Versatile – Used in web apps, AI, ML, data science, IoT, automation, etc.""",

 "data/text_files/ml_intro":"""Machine Learning (ML) is a branch of Artificial Intelligence (AI) where systems learn from data and improve their performance automatically without being explicitly programmed for every task.

Instead of writing rules, you provide the machine with data + algorithms, and it figures out patterns, makes predictions, or decisions.""",

 "data/text_files/AI_intro":"""Artificial Intelligence (AI) is a branch of computer science that deals with creating machines or software capable of performing tasks that normally require human intelligence. 
 These tasks include reasoning, learning, decision making, problem solving, understanding language, and perception.
John McCarthy (who coined the term in 1956) defined AI as “the science and engineering of making intelligent machines.”"""

}

In [6]:
for filepath,content in sample_data.items():
    with open(filepath,'w',encoding='utf-8') as f:
        f.write(content)

read the file using TextLoader - Single File

In [27]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/Users/home/Documents/MR/RAG/Data_Ingestion_Parsing/data/text_files/python_intro.txt",encoding="utf-8")

Documents = loader.load()
# print(Documents)
# print("---------")
od = Documents[0]#both page content & metadata respectively
print(od)
print(od.page_content)
print("-------==========-")
print(od.metadata['source'])
print(Documents[0])
# print("---------=========")
# # print("---",Documents.metadata.get('source','unknown'))
# print("---",Documents.metadata['source'])
# print("---",Documents)
# print('oooooo')
# print(doc.page_content)   # full text of the file
 # full text of the file)

print(type(Documents))
print(f"loaded {len(Documents)} documents")
print(f"content preview:{Documents[0].page_content[:100]}....")
print(f"metadata: {Documents[0].metadata}")


page_content='Python is a high-level, interpreted, object-oriented programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

⚡ Key Features of Python

Simple & Readable – Easy syntax, close to English.

Interpreted – No need to compile; runs line by line.

Cross-Platform – Works on Windows, macOS, Linux, etc.

Object-Oriented – Supports classes, objects, inheritance, etc.

Dynamic Typing – No need to declare variable types.

Large Standard Library – Comes with modules for math, OS, web, AI, etc.

Community Support – Huge developer community and resources.

Versatile – Used in web apps, AI, ML, data science, IoT, automation, etc.' metadata={'source': '/Users/home/Documents/MR/RAG/Data_Ingestion_Parsing/data/text_files/python_intro.txt'}
Python is a high-level, interpreted, object-oriented programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

⚡ Key

<!-- load all the text files from the directory
 -->


In [30]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader

dir_loader = DirectoryLoader(
    "data/text_files", #path
    glob="**/*.txt", #pattern to match files, ** = folder , * = file
    loader_cls = TextLoader,
    loader_kwargs = {'encoding':'utf-8'}, #means the loader (reader) will open and read the file using UTF-8 encoding, which allows it to correctly understand and display: English letters,Numbers & punctuation,Accented characters,Non-Latin scripts,Emoji 
    show_progress = True # loading progress

)

documents = dir_loader.load()

print(f"loaded {len(documents)} documents\n")


for i,doc in enumerate(documents):
    print(f"document {i+1}")
    # print(doc) # return page content(content of the file) and metadata(contain source[the relative path])
    print(f"source : {doc.metadata}")
    print(f"source : {doc.metadata['source']}")
    # print("***********")
    
    print(f"length :{len(doc.page_content)} characters")
    print("-------------")
    # print(f"length :{doc.page_content} characters")


100%|██████████| 3/3 [00:00<00:00, 2046.33it/s]

loaded 3 documents

document 1
source : {'source': 'data/text_files/python_intro.txt'}
source : data/text_files/python_intro.txt
length :691 characters
-------------
document 2
source : {'source': 'data/text_files/AI_intro.txt'}
source : data/text_files/AI_intro.txt
length :413 characters
-------------
document 3
source : {'source': 'data/text_files/ml_intro.txt'}
source : data/text_files/ml_intro.txt
length :327 characters
-------------


Chunking is not just splitting text—it’s an optimization problem balancing context, retrieval accuracy, and efficiency.

Yes ✅ — overlap in chunking is definitely beneficial in RAG

1. Why Overlap Helps

Preserves continuity → If an important idea is split between two chunks, overlap ensures both chunks have enough context.

Improves retrieval → Search engines (vector or keyword) can match a query to either chunk without missing part of the answer.

Reduces hallucination → LLM doesn’t get isolated fragments that don’t make sense alone.

Exmaple - Chunk A: "The contract requires both parties to comply with..."
          Chunk B: "...comply with all safety regulations. Failure to do so..."   
          
          👉 Now each chunk makes sense independently.

🔹 2. When Overlap is Most Useful

When chunks are cut mid-sentence or mid-paragraph.

For legal, medical, or technical docs where small missing context can change meaning.

When using embedding/vector search, since embeddings work better if chunks are self-contained.

🔹 3. How Much Overlap?

Small overlap (50–100 tokens) is usually enough.

Too little overlap → risk of losing context.

Too much overlap → duplicates increase storage cost and slow retrieval.

👉 Sweet spot: ~10–20% of chunk size.
Example:

Chunk size = 500 tokens → Overlap = 50–100 tokens.

Text Splitting Stratergies

In [32]:
from langchain.text_splitter import(
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter
)

print(documents)

[Document(metadata={'source': 'data/text_files/python_intro.txt'}, page_content='Python is a high-level, interpreted, object-oriented programming language known for its simplicity and readability.\nIt was created by Guido van Rossum and first released in 1991.\n\n⚡ Key Features of Python\n\nSimple & Readable – Easy syntax, close to English.\n\nInterpreted – No need to compile; runs line by line.\n\nCross-Platform – Works on Windows, macOS, Linux, etc.\n\nObject-Oriented – Supports classes, objects, inheritance, etc.\n\nDynamic Typing – No need to declare variable types.\n\nLarge Standard Library – Comes with modules for math, OS, web, AI, etc.\n\nCommunity Support – Huge developer community and resources.\n\nVersatile – Used in web apps, AI, ML, data science, IoT, automation, etc.'), Document(metadata={'source': 'data/text_files/AI_intro.txt'}, page_content='Artificial Intelligence (AI) is a branch of computer science that deals with creating machines or software capable of performing 

method-1 Character - based splitting

In [ ]:
text = documents[0].page_content # in document[0] both the page content and metadata present

char_splitter = CharacterTextSplitter(
    separator="\n", #split on new lines
    chunk_size = 200, #max chunk size in character
    chunk_overlap = 20, #overlap between chunks
    length_function = len # how to measure the chunk size 
)

char_chunks = char_splitter.split_text(text)
print(f"Created {len(char_chunks)} chunks")

print(f"First chunk: {char_chunks[0][:100]}")
print("--------")

print(char_chunks[0])
print("--------------")

print(char_chunks[1])
print("--------------")

print(char_chunks[2])
print("--------------")

print(char_chunks[3])
print("--------------")


Created 4 chunks
First chunk: Python is a high-level, interpreted, object-oriented programming language known for its simplicity a
--------
Python is a high-level, interpreted, object-oriented programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.
--------------
⚡ Key Features of Python
Simple & Readable – Easy syntax, close to English.
Interpreted – No need to compile; runs line by line.
Cross-Platform – Works on Windows, macOS, Linux, etc.
--------------
Object-Oriented – Supports classes, objects, inheritance, etc.
Dynamic Typing – No need to declare variable types.
Large Standard Library – Comes with modules for math, OS, web, AI, etc.
--------------
Community Support – Huge developer community and resources.
Versatile – Used in web apps, AI, ML, data science, IoT, automation, etc.
--------------


method-2 Recursive Character Text Splitter (Recommended)

In [ ]:
text = documents[0].page_content

recursive_splitter = RecursiveCharacterTextSplitter(
    # separators=["\n","\n\n",","," "], #here overlaping cannot be see
    separators=" ", # here overlaping can be see because of " " space splitter
    chunk_size = 200,
    chunk_overlap = 20,
    length_function = len
)

recursive_chunks = recursive_splitter.split_text(text)
print(f"Created {len(recursive_chunks)} chunks")

print(f"First chunk: {recursive_chunks[0][:100]}")
print("--------")

print(recursive_chunks[0])
print("--------------")

print(recursive_chunks[1])
print("--------------")

print(recursive_chunks[2])
print("--------------")

print(recursive_chunks[3])
print("--------------")


Created 4 chunks
First chunk: Python is a high-level, interpreted, object-oriented programming language known for its simplicity a
--------
Python is a high-level, interpreted, object-oriented programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

⚡ Key Features of
--------------
Key Features of Python

Simple & Readable – Easy syntax, close to English.

Interpreted – No need to compile; runs line by line.

Cross-Platform – Works on Windows, macOS, Linux,
--------------
macOS, Linux, etc.

Object-Oriented – Supports classes, objects, inheritance, etc.

Dynamic Typing – No need to declare variable types.

Large Standard Library – Comes with modules for math, OS, web,
--------------
for math, OS, web, AI, etc.

Community Support – Huge developer community and resources.

Versatile – Used in web apps, AI, ML, data science, IoT, automation, etc.
--------------


Method-3 Token - based splitting

In [94]:
text = documents[0].page_content

token_splitter = TokenTextSplitter(
    
    chunk_size = 50,
    chunk_overlap = 10,
    # length_function = len
)

token_chunks = token_splitter.split_text(text)
print(f"Created {len(token_chunks)} chunks")

print(f"First chunk: {token_chunks[0][:100]}")
print("--------")

print(token_chunks[0])
print("--------------")

print(token_chunks[1])
print("--------------")

print(token_chunks[2])
print("--------------")

print(token_chunks[3])
print("--------------")


Created 5 chunks
First chunk: Python is a high-level, interpreted, object-oriented programming language known for its simplicity a
--------
Python is a high-level, interpreted, object-oriented programming language known for its simplicity and readability.
It was created by Guido van Rossum and first released in 1991.

⚡ Key Features of Python

Simple
--------------
⚡ Key Features of Python

Simple & Readable – Easy syntax, close to English.

Interpreted – No need to compile; runs line by line.

Cross-Platform – Works on Windows, macOS, Linux
--------------
-Platform – Works on Windows, macOS, Linux, etc.

Object-Oriented – Supports classes, objects, inheritance, etc.

Dynamic Typing – No need to declare variable types.

Large Standard Library – Comes
--------------
 variable types.

Large Standard Library – Comes with modules for math, OS, web, AI, etc.

Community Support – Huge developer community and resources.

Versatile – Used in web apps, AI, ML, data science
--------------
